In [1]:
import pandas as pd
import numpy as np

In [2]:
death = pd.read_csv(
    r"C:\PDS\opioid-2023-kml\00_Data\Deaths_Data\state_text_files\drug_deaths.txt",
    delimiter="\t",
)
population = pd.read_csv(
    r"C:\PDS\opioid-2023-kml\00_Data\Population_Data\population.csv"
)
population.head(5)
death["County Code"] = pd.to_numeric(
    death["County Code"], errors="coerce"
)  # Convert to numeric
death["County Code"] = death["County Code"].astype(int)
import math


death["State Code"] = death["County Code"].apply(lambda x: float(math.floor(x / 1000)))


unique_state_code = death["State Code"].unique()


print(unique_state_code)
subset_population = population[population["State_FIPS"].isin(unique_state_code)]
print(death["Year"].min())
print(death["Year"].max())
subset_population = subset_population.drop(["2002", "2016"], axis=1)
subset_population
print(subset_population["FIPS"].nunique())
print(death["County Code"].nunique())
subset_population.head(10)
death = death.rename(columns={"County Code": "FIPS"})
subset_population.columns
columns_to_melt = [
    "2003",
    "2004",
    "2005",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
]  # update the list base on your year
melted_df = pd.melt(
    subset_population,
    id_vars=["FIPS", "State", "County_FIPS"],
    value_vars=columns_to_melt,
    var_name="Year",
    value_name="Population",
)
melted_df["Year"] = melted_df["Year"].astype(int)
melted_df
merged_df = pd.merge(death, melted_df, on=["Year", "FIPS"], how="outer")
merged_df
merged_df = merged_df.drop(columns=["State Code", "County"])

[17. 39. 12. 47. 48. 53. 54. 20. 41. 31. 56.]
2003
2015
957
312


In [3]:
merged_df

,Year,FIPS,Deaths,State,County_FIPS,Population
0,2015,17001,14.0,IL,1,66850.0
1,2012,39001,10.0,OH,1,28288.0
2,2014,39001,10.0,OH,1,28060.0
3,2015,39001,13.0,OH,1,27919.0
4,2003,12001,11.0,FL,1,227022.0
...,...,...,...,...,...,...
12436,2015,56037,NaN,WY,37,44780.0
12437,2015,56039,NaN,WY,39,23083.0
12438,2015,56041,NaN,WY,41,20777.0
12439,2015,56043,NaN,WY,43,8282.0


In [4]:
death_data = pd.read_parquet(
    r"C:\PDS\opioid-2023-kml\00_Data\Deaths_Data\filled_deaths.parquet"
)

In [5]:
death_data = death_data.drop(
    columns=["MME_Conversion_Factor", "per_capita_MME_mg", "County_FIPS"]
)

In [6]:
has_nan = death_data.isna().any().any()

if has_nan:
    print("The DataFrame has NaN values.")
else:
    print("The DataFrame does not have any NaN values.")

The DataFrame does not have any NaN values.


In [7]:
death_data[death_data["State"] == "TX"]

,Year,FIPS,Deaths_df1,State,Population
4857,2006,48001,10.0,TX,57386.0
4858,2007,48001,0.0,TX,57870.0
4859,2008,48001,10.0,TX,57963.0
4860,2009,48001,0.0,TX,58410.0
4861,2010,48001,0.0,TX,58498.0
...,...,...,...,...,...
7113,2011,48507,0.0,TX,11872.0
7114,2012,48507,0.0,TX,12015.0
7115,2013,48507,0.0,TX,12203.0
7116,2014,48507,0.0,TX,12254.0


In [8]:
new_df = pd.merge(death_data, merged_df, on=["FIPS", "Year"], how="right")

In [9]:
subset_df = new_df[(pd.notnull(new_df["Deaths"]) | pd.notnull(new_df["Deaths_df1"]))]

In [10]:
subset_df["Deaths"] = subset_df["Deaths_df1"].combine_first(subset_df["Deaths"])

C:\Users\khsqu\AppData\Local\Temp\ipykernel_4132\3745002463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["Deaths"] = subset_df["Deaths_df1"].combine_first(subset_df["Deaths"])


In [11]:
subset_df["Population"] = subset_df["Population_x"].combine_first(
    subset_df["Population_y"]
)

C:\Users\khsqu\AppData\Local\Temp\ipykernel_4132\2981538775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["Population"] = subset_df["Population_x"].combine_first(


In [12]:
subset_df["State"] = subset_df["State_x"].combine_first(subset_df["State_y"])

C:\Users\khsqu\AppData\Local\Temp\ipykernel_4132\1596900169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["State"] = subset_df["State_x"].combine_first(subset_df["State_y"])


In [13]:
subset_df = subset_df.loc[:, ["Year", "Population", "Deaths", "State", "FIPS"]]

In [14]:
subset_df

,Year,Population,Deaths,State,FIPS
0,2015,66850.0,14.0,IL,17001
1,2012,28288.0,10.0,OH,39001
2,2014,28060.0,10.0,OH,39001
3,2015,27919.0,13.0,OH,39001
4,2003,227022.0,11.0,FL,12001
...,...,...,...,...,...
12436,2015,44780.0,13.0,WY,56037
12437,2015,23083.0,1.0,WY,56039
12438,2015,20777.0,8.0,WY,56041
12439,2015,8282.0,12.0,WY,56043


In [15]:
missing_deaths_df = new_df[new_df["Deaths"].isnull() & new_df["Deaths_df1"].isnull()]

In [16]:
missing_deaths_df["State"] = missing_deaths_df["State_x"].combine_first(
    missing_deaths_df["State_y"]
)

C:\Users\khsqu\AppData\Local\Temp\ipykernel_4132\582006316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_deaths_df["State"] = missing_deaths_df["State_x"].combine_first(


In [17]:
missing_deaths_df["Population"] = missing_deaths_df["Population_x"].combine_first(
    missing_deaths_df["Population_y"]
)

C:\Users\khsqu\AppData\Local\Temp\ipykernel_4132\3204613154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_deaths_df["Population"] = missing_deaths_df["Population_x"].combine_first(


In [18]:
missing_deaths_df = missing_deaths_df.loc[
    :, ["Year", "Population", "Deaths", "State", "FIPS"]
]

In [19]:
missing_deaths_df

,Year,Population,Deaths,State,FIPS
2338,2003,23555.0,NaN,FL,12003
2339,2003,27035.0,NaN,FL,12007
2340,2003,13293.0,NaN,FL,12013
2341,2003,153235.0,NaN,FL,12015
2342,2003,59395.0,NaN,FL,12023
...,...,...,...,...,...
12315,2015,906.0,NaN,TX,48393
12325,2015,3191.0,NaN,TX,48413
12333,2015,1330.0,NaN,TX,48431
12337,2015,863.0,NaN,TX,48443


* Regression imputation for texas 2003-2005



In [20]:
# using merged_df we will begin the stochastic imputation process.
# stochastic imputation will be in a function so that we can test it multiple times.

from sklearn.linear_model import LinearRegression


def train_regression_model(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model


def preprocess_data(df, predictors, target):
    # Extract numerical predictors (excluding target and categorical variables)
    numerical_predictors = [
        col for col in predictors if col != target and df[col].dtype != "object"
    ]

    # One-hot encode categorical variables
    categorical_predictors = [
        col for col in predictors if col != target and col not in numerical_predictors
    ]
    df_encoded = pd.get_dummies(df, columns=categorical_predictors, drop_first=True)

    return df_encoded

In [21]:
predictors = ["Year", "State", "FIPS"]
# dfs = subset_df and missing_deaths_df
# target = "Deaths"

In [22]:
encoded_filled = preprocess_data(subset_df, predictors, "Deaths")

In [23]:
encoded_filled

,Year,Population,Deaths,FIPS,State_IL,State_KS,State_NE,State_OH,State_OR,State_TN,State_TX,State_WA,State_WV,State_WY
0,2015,66850.0,14.0,17001,True,False,False,False,False,False,False,False,False,False
1,2012,28288.0,10.0,39001,False,False,False,True,False,False,False,False,False,False
2,2014,28060.0,10.0,39001,False,False,False,True,False,False,False,False,False,False
3,2015,27919.0,13.0,39001,False,False,False,True,False,False,False,False,False,False
4,2003,227022.0,11.0,12001,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12436,2015,44780.0,13.0,56037,False,False,False,False,False,False,False,False,False,True
12437,2015,23083.0,1.0,56039,False,False,False,False,False,False,False,False,False,True
12438,2015,20777.0,8.0,56041,False,False,False,False,False,False,False,False,False,True
12439,2015,8282.0,12.0,56043,False,False,False,False,False,False,False,False,False,True


In [24]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(
    encoded_filled, test_size=0.10, random_state=42
)

In [25]:
model = train_regression_model(train_data.drop("Deaths", axis=1), train_data["Deaths"])

In [26]:
predicted_values_test = model.predict(test_data.drop("Deaths", axis=1))

In [27]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

mse_test = mean_squared_error(test_data["Deaths"], predicted_values_test)
r2_test = r2_score(test_data["Deaths"], predicted_values_test)
print(f"Mean Squared Error on Testing Set: {mse_test}")
print(f"R2_Score on Testing Set: {r2_test}")

Mean Squared Error on Testing Set: 325.2915825387634
R2_Score on Testing Set: 0.8363251345754171


In [28]:
missing_data_encoded = preprocess_data(missing_deaths_df, predictors, "Deaths")

predicted_values_missing = model.predict(missing_data_encoded.drop("Deaths", axis=1))

# Impute the missing values in the original dataset with the predicted values
missing_data_encoded.loc[missing_data_encoded.index, "Deaths"] = np.clip(
    predicted_values_missing, 0, None
)

In [29]:
missing_data_encoded["Deaths"] = np.round(missing_data_encoded["Deaths"]).astype(int)

In [30]:
missing_data_encoded

,Year,Population,Deaths,FIPS,State_IL,State_KS,State_NE,State_OH,State_OR,State_TN,State_TX,State_WA,State_WV,State_WY
2338,2003,23555.0,5,12003,False,False,False,False,False,False,False,False,False,False
2339,2003,27035.0,6,12007,False,False,False,False,False,False,False,False,False,False
2340,2003,13293.0,4,12013,False,False,False,False,False,False,False,False,False,False
2341,2003,153235.0,20,12015,False,False,False,False,False,False,False,False,False,False
2342,2003,59395.0,9,12023,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12315,2015,906.0,0,48393,False,False,False,False,False,False,True,False,False,False
12325,2015,3191.0,0,48413,False,False,False,False,False,False,True,False,False,False
12333,2015,1330.0,0,48431,False,False,False,False,False,False,True,False,False,False
12337,2015,863.0,0,48443,False,False,False,False,False,False,True,False,False,False


In [31]:
state_columns = [
    "State_IL",
    "State_KS",
    "State_NE",
    "State_OH",
    "State_OR",
    "State_TN",
    "State_TX",
    "State_WA",
    "State_WV",
    "State_WY",
]


def get_state(row):
    for state_column in state_columns:
        if row[state_column]:
            return state_column.split("_")[1]
    # If none of the state columns are True, default to 'FL'
    return "FL"


# Create a new 'State' column using the apply function
missing_data_encoded["State"] = missing_data_encoded.apply(get_state, axis=1)

# Display the updated DataFrame
missing_data_encoded

,Year,Population,Deaths,FIPS,State_IL,State_KS,State_NE,State_OH,State_OR,State_TN,State_TX,State_WA,State_WV,State_WY,State
2338,2003,23555.0,5,12003,False,False,False,False,False,False,False,False,False,False,FL
2339,2003,27035.0,6,12007,False,False,False,False,False,False,False,False,False,False,FL
2340,2003,13293.0,4,12013,False,False,False,False,False,False,False,False,False,False,FL
2341,2003,153235.0,20,12015,False,False,False,False,False,False,False,False,False,False,FL
2342,2003,59395.0,9,12023,False,False,False,False,False,False,False,False,False,False,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12315,2015,906.0,0,48393,False,False,False,False,False,False,True,False,False,False,TX
12325,2015,3191.0,0,48413,False,False,False,False,False,False,True,False,False,False,TX
12333,2015,1330.0,0,48431,False,False,False,False,False,False,True,False,False,False,TX
12337,2015,863.0,0,48443,False,False,False,False,False,False,True,False,False,False,TX


In [32]:
filled_data_merge = missing_data_encoded.drop(columns=state_columns)

# Display the updated DataFrame
filled_data_merge

,Year,Population,Deaths,FIPS,State
2338,2003,23555.0,5,12003,FL
2339,2003,27035.0,6,12007,FL
2340,2003,13293.0,4,12013,FL
2341,2003,153235.0,20,12015,FL
2342,2003,59395.0,9,12023,FL
...,...,...,...,...,...
12315,2015,906.0,0,48393,TX
12325,2015,3191.0,0,48413,TX
12333,2015,1330.0,0,48431,TX
12337,2015,863.0,0,48443,TX


In [33]:
encoded_filled["State"] = encoded_filled.apply(get_state, axis=1)
filled_merge = encoded_filled.drop(columns=state_columns)
filled_merge

,Year,Population,Deaths,FIPS,State
0,2015,66850.0,14.0,17001,IL
1,2012,28288.0,10.0,39001,OH
2,2014,28060.0,10.0,39001,OH
3,2015,27919.0,13.0,39001,OH
4,2003,227022.0,11.0,12001,FL
...,...,...,...,...,...
12436,2015,44780.0,13.0,56037,WY
12437,2015,23083.0,1.0,56039,WY
12438,2015,20777.0,8.0,56041,WY
12439,2015,8282.0,12.0,56043,WY


In [34]:
export_data = pd.concat([filled_merge, filled_data_merge])

In [35]:
export_data

,Year,Population,Deaths,FIPS,State
0,2015,66850.0,14.0,17001,IL
1,2012,28288.0,10.0,39001,OH
2,2014,28060.0,10.0,39001,OH
3,2015,27919.0,13.0,39001,OH
4,2003,227022.0,11.0,12001,FL
...,...,...,...,...,...
12315,2015,906.0,0.0,48393,TX
12325,2015,3191.0,0.0,48413,TX
12333,2015,1330.0,0.0,48431,TX
12337,2015,863.0,0.0,48443,TX


In [36]:
export_data.to_parquet(
    r"C:\PDS\opioid-2023-kml\00_Data\Deaths_Data\export_data.parquet", index=False
)